In [4]:
import os
import json
import time
from IPython.display import Markdown, display
import requests
import textwrap
import boto3
from utils import read_file, save_file
from dotenv import load_dotenv

In [5]:
load_dotenv()
ab_paper = read_file('data/whitepaper/AB_2013-07_Model_Risk_Management_Guidance.md')
moody_paper = read_file('data/whitepaper/riskcalc-3.1-whitepaper.md')

In [6]:
br = boto3.client(service_name='bedrock')
model_summaries = br.list_foundation_models()['modelSummaries']
#print(json.dumps(model_summaries, indent=4))

In [10]:
cloude = br.get_foundation_model(modelIdentifier='anthropic.claude-3-haiku-20240307-v1:0')['modelDetails']
print(json.dumps(cloude, indent=4))

{
    "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0",
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "modelName": "Claude 3 Haiku",
    "providerName": "Anthropic",
    "inputModalities": [
        "TEXT",
        "IMAGE"
    ],
    "outputModalities": [
        "TEXT"
    ],
    "responseStreamingSupported": true,
    "customizationsSupported": [],
    "inferenceTypesSupported": [
        "ON_DEMAND"
    ],
    "modelLifecycle": {
        "status": "ACTIVE"
    }
}


In [51]:
def get_document_analysis_claude(document, question, model='anthropic.claude-3-haiku-20240307-v1:0', temperature=0, tokens=3000, top_p=0.9, top_k=250):
    instructions = f"""As a model risk manager, please conduct a detailed model analysis.
You are provided with model whitepaper to perform the analysis.
You are provided with analysis objective in the user input.
Your response should use well formatted Markdown forman and include:
Section with specific findings including numbered references from the references section. 
Secton with numbered references to the whitepaper, citing specific sections, tables, or figures that support the findings printed in numbered order. 
A clear, evidence-based recommendation section on whether the model should be adopted for usage, given the findings.
<context>
{document}
</context>
"""
    
    brt = boto3.client(service_name='bedrock-runtime')
    
    body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "system": instructions,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": question
                }
            ]
        }
    ],
    "max_tokens": tokens,
    "temperature": temperature,
    "top_p": top_p,
    "top_k": top_k
    })

    accept = 'application/json'
    contentType = 'application/json'

    response = brt.invoke_model(body=body, modelId=model, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0]['text']

In [52]:
qq = ['Identify any specific limitations and model usage risk in stagflation environment',
      'Indentify any specific limitations and model usage risks in hyper-inflation scenario']

model = 'anthropic.claude-3-haiku-20240307-v1:0'
#model = 'anthropic.claude-3-sonnet-20240229-v1:0'
for i, q in enumerate(qq):
    content = get_document_analysis_claude(moody_paper, q, model=model, tokens=4096)
    title = (f"## {q.capitalize()}")
    display(Markdown(title))
    display(Markdown(content))
    #save_file(f"reports/moody-risk-calc-analysis-cloude-21-{i+1}.md", f"{title}\n{content}")


## Identify any specific limitations and model usage risk in stagflation environment

Based on the model whitepaper, here are the key findings regarding limitations and model usage risks in a stagflation environment:

Findings:

1. The RiskCalc v3.1 model incorporates industry-specific and economy-wide market information, which helps capture systematic risk factors. However, the model may not fully account for the complex dynamics of a stagflation environment, where high inflation and low growth coexist.

2. The model's ability to stress test firms from different points in the economic cycle is highlighted. However, the whitepaper notes that "there is no way to predict the impact that every unexpected event might have on a borrower's loan performance." A stagflation scenario may present unique challenges that the model has not been fully tested against.

3. The model's performance is validated using data up to 2002, which may not capture the nuances of a more recent stagflation environment. The whitepaper acknowledges the model's limitations in predicting the impact of "unexpected events" on borrowers.

4. The model's reliance on financial statement data, which can lag market conditions, may be a limitation in a rapidly changing stagflation environment. The forward-looking market-based factors included in the model may not fully compensate for this lag.

References:

1. Section 2.3 "Support for Regulatory Requirements" discusses the model's ability to stress test default probabilities, but notes the limitations in predicting the impact of "unexpected events."
2. Section 3.4.1 "Managing Data Quality" acknowledges the potential for misclassification errors and questionable accounting practices, which may be exacerbated in a stagflation environment.
3. Section 4 "Model Validation" highlights the model's performance testing, but the data used may not fully capture a stagflation scenario.

Recommendation:

Given the potential limitations of the RiskCalc v3.1 model in a stagflation environment, users should exercise caution and closely monitor the model's performance. Additional stress testing and validation specific to stagflation conditions may be warranted to fully understand the model's capabilities and limitations in such an environment.

## Indentify any specific limitations and model usage risks in hyper-inflation scenario

Based on the whitepaper, here are the key limitations and model usage risks in a hyperinflation scenario:

1. Limitations in Handling Extreme Economic Conditions:
   - The whitepaper notes that the RiskCalc v3.1 model was developed and validated using data up to 2002, which included the "volatility jump of 1998-1999". 
   - However, the model's performance during more extreme economic conditions, such as a hyperinflation scenario, is not explicitly addressed.
   - The ability of the model to accurately predict default risk during periods of hyperinflation is uncertain, as the underlying data and economic relationships may not capture such extreme conditions.

2. Potential Instability of Financial Ratios:
   - During hyperinflation, standard financial ratios used as inputs to the model (e.g. leverage, profitability, liquidity) may become unstable and less informative.
   - Rapid changes in prices, currency values, and other macroeconomic factors can distort the meaning and interpretation of these financial metrics, potentially reducing the model's predictive power.

3. Limitations of the Distance-to-Default Measure:
   - The RiskCalc v3.1 model incorporates the distance-to-default measure, which relies on equity market information to capture systematic risk.
   - In a hyperinflation scenario, equity markets may become highly volatile and disconnected from underlying firm fundamentals, undermining the validity of the distance-to-default measure as a leading indicator of default risk.

4. Potential Breakdown of Historical Relationships:
   - The model is built on historical relationships between financial ratios, industry factors, and default rates.
   - During a hyperinflation event, these historical relationships may break down, as the economic environment deviates significantly from the conditions used to develop and validate the model.
   - This could lead to the model producing inaccurate default probability estimates that do not reflect the true risk profile of borrowers.

5. Data Availability and Quality Concerns:
   - The whitepaper emphasizes the importance of the extensive Credit Research Database used to develop the RiskCalc v3.1 model.
   - However, the availability and quality of financial data for private firms during a hyperinflation scenario may be severely compromised, potentially undermining the model's predictive capabilities.

In summary, the key limitations and risks in using the RiskCalc v3.1 model in a hyperinflation scenario include the model's potential inability to accurately capture extreme economic conditions, the instability of financial ratios, the limitations of the distance-to-default measure, the breakdown of historical relationships, and concerns about data availability and quality. Caution should be exercised when applying the model in such an environment, and additional validation and adjustments may be necessary to ensure the model's reliability.

In [35]:
qq = ["Generate a list of the model analysis objectives using provided model whitepaper",
      "Generate a list of objecttives for assessing the model peformance in different economic environments usingf providedl model whitepaper"]

for i, q in enumerate(qq):
    content = get_document_analysis_claude(moody_paper, q, model=model, tokens=4096)
    title = (f"## {q.capitalize()}")
    display(Markdown(title))
    display(Markdown(content))
    save_file(f"reports/moody-riskcalc-objectives-claude-{i+1}.md", f"{title}\n{content}")

## Generate a list of the model analysis objectives using provided model whitepaper

Here is a detailed model analysis focusing on the objectives using the provided model whitepaper:

## Model Analysis Objectives

Based on the whitepaper, the key objectives of the RiskCalc v3.1 model analysis are:

**1. Assess the strategic assets and innovations of the RiskCalc v3.1 model:**
- Evaluate the benefits of the expanded data pool for predictions, including the size and quality improvements to the Credit Research Database.
- Examine how the model supports regulatory requirements under the New Basel Capital Accord, such as providing consistent risk estimates, forward-looking risk ratings, stress testing default probabilities, and robust model validation.

**2. Understand the model framework and key features:**
- Examine the Financial Statement Only (FSO) mode and how it selects a limited set of financial ratios to avoid overfitting.
- Analyze the complete RiskCalc v3.1 model, including how it blends firm-specific financial data with forward-looking market information through the distance-to-default measure.
- Evaluate the importance of controlling for industry variation in the model.

**3. Assess the model validation and performance:**
- Review the out-of-sample and out-of-time testing approaches, including K-fold analysis and walk-forward testing, to evaluate model power and calibration.
- Examine the model's performance over the credit cycle and compare it to alternative models like the Private Firm Model and Z-score.

**4. Evaluate the economic value of the RiskCalc v3.1 model:**
- Analyze the potential profit and loss benefits for lenders using the RiskCalc v3.1 model compared to alternative models.

## Key Findings from the Model Analysis

1. **Strategic Assets and Innovations:**
   - The RiskCalc v3.1 model is built on the world's largest and cleanest private company default database, the Moody's KMV Credit Research Database, which has over 6.5 million financial statements and 97,000 default events globally.
   - The model is designed to meet the requirements of the New Basel Capital Accord, including providing consistent risk estimates, forward-looking risk ratings, the ability to stress test default probabilities, and robust model validation.

2. **Model Framework and Key Features:**
   - The Financial Statement Only (FSO) mode selects a limited set of financial ratios to capture profitability, leverage, debt coverage, growth, liquidity, activity, and size, avoiding overfitting by using non-parametric transformations.
   - The complete RiskCalc v3.1 model blends firm-specific financial data with forward-looking market information through the distance-to-default measure, which captures systematic industry and economic trends.
   - Controlling for industry variation is crucial, as it allows the model to account for differences in default rates and the interpretation of financial ratios across sectors.

3. **Model Validation and Performance:**
   - Rigorous out-of-sample and out-of-time testing, including K-fold analysis and walk-forward testing, demonstrates the model's strong predictive power and calibration, outperforming alternatives like the Private Firm Model and Z-score.
   - The model maintains its performance over the credit cycle, with the complete RiskCalc v3.1 model showing significantly higher accuracy ratios compared to previous versions and other models.

4. **Economic Value:**
   - Analyses suggest the RiskCalc v3.1 model can provide substantial economic benefits for lenders, potentially increasing the profitability of a loan portfolio by up to 25 basis points compared to alternatives.
   - In a competitive environment, a medium-sized bank using the RiskCalc v3.1 model could see profits increase by more than $10 million on average compared to a competitor using a weaker model like Z-score.

## Recommendation

Based on the comprehensive analysis of the RiskCalc v3.1 model, I would recommend adopting this model for usage. The model demonstrates significant improvements in predictive power, calibration, and economic value compared to previous versions and alternative models. The rigorous validation testing, adherence to regulatory requirements, and the model's ability to capture both firm-specific and systematic risk factors make it a compelling choice for lenders seeking to improve their credit risk assessment and portfolio management. The potential profit and loss benefits of using the RiskCalc v3.1 model are substantial and could provide a meaningful competitive advantage in the market.

## Generate a list of objecttives for assessing the model peformance in different economic environments usingf providedl model whitepaper

Here is a detailed model analysis focusing on assessing the model performance in different economic environments using the provided model whitepaper:

**Bullet points highlighting specific analysis topics:**

- The RiskCalc v3.1 model is designed to meet the requirements of the New Basel Capital Accords, including extensive documentation, validation, and testing. This allows the model to provide consistent risk estimates, forward-looking risk ratings, and the ability to stress test default probabilities under different economic conditions (Section 2.3).
- The model's performance is validated using rigorous out-of-sample and out-of-time testing, including K-fold analysis and walk-forward analysis. These tests demonstrate the model's stability and superior predictive power compared to previous versions and other models like the Private Firm Model and Z-score (Section 4).
- The model's performance is shown to be robust across the credit cycle, maintaining high accuracy ratios even during periods of high default activity (Section 4.4).
- The expanded data pool used to develop the RiskCalc v3.1 model, including data from the 2000-2002 period of high default rates, allows the model to be calibrated to a wide range of general credit cycle conditions and to stress test the impact of a changing economy on default likelihoods (Section 2.2).
- The incorporation of industry-specific information and market-based factors (distance-to-default) into the model allows it to better capture systematic, sector-based risks that may not be fully reflected in financial statements alone. This improves the model's performance, particularly over shorter horizons when market information leads the accounting data (Sections 3.2 and 3.3).

**Recommendation:**

Based on the detailed analysis provided in the whitepaper, the RiskCalc v3.1 model appears to be a robust and high-performing model for assessing the credit risk of private, middle-market companies. The model's design to meet regulatory requirements, its superior predictive power demonstrated through rigorous validation, and its ability to capture systematic and idiosyncratic risks across different economic environments make it a strong candidate for adoption.

The whitepaper provides compelling evidence that the RiskCalc v3.1 model outperforms previous versions as well as other widely used models like the Private Firm Model and Z-score. The model's stress testing capabilities and responsiveness to changing market conditions are also valuable features that allow users to assess credit risk under a variety of economic scenarios.

Overall, the detailed analysis presented in the whitepaper suggests that the RiskCalc v3.1 model should be strongly considered for adoption, as it appears to be a significant improvement over previous models and well-suited for assessing private firm credit risk in different economic environments.